### In this notebook we will cover how to compute the skew metric from the paper ["What Does Individual Option Volatility Smirk Tell Us About Future Equity Returns?"](http://papers.ssrn.com/sol3/papers.cfm?abstract_id=1107464) by Yuhang Xing, Xiaoyan Zhang and Rui Zhao using `dask` and `pandas`. 

### Why Dask and Not Just Pandas?

If you are not familiar with the package, `dask` is designed to provide easy parallelization of code on a single machine or across clusters. The primary benefit of `dask`, for most users, is that it provides tools to work on data that is __too big__ to fit __in-memory__. 

If you have been a `pandas` user for any length of time you likely will have encountered __memory errors__, sometimes, even in situations where you _should_ have enough memory available on your system. The primary creator of `pandas`, Wes McKinney has stated that the memory handling is one of the main pain points of his creation, and until such time that another tool(set) can be developed to replace it, `dask` does an admirable job of alleviating these issues for us. 

The intuition behind how `dask` works with `pandas` can be likened to the __split-apply-combine__ logic of the groupby function. Dask intelligently splits up the data into blocks, applies transformations and operations, and then combines them for a final result. To boost the performance of this process it creates a __task graph__. The task graph does the job of organizing the tasks in such a manner as to cache intermediate results that apply to all the blocks, so that they don't have to be recalculated/reprocessed unnecessarily and to provide concurrency to other tasks where appropriate. 

What makes it particularly attractive for `pandas` users is that it borrows much of the syntax and api directly from `pandas` which lowers the learning curve tremendously. 

In [1]:
%load_ext watermark
%watermark

import sys
import os

data_dir = '~/option_skew_project_public/data/'
script_dir = '~/option_skew_project_public/scripts/'
sys.path.append(script_dir)
import pq_save

#import dask.dataframe as dd
from dask import delayed, dataframe as dd
from dask.diagnostics import ProgressBar
pbar = ProgressBar(); pbar.register()

import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

pd.options.display.float_format = '{:,.4f}'.format
import numpy as np
import scipy.stats as stats

from mpl_toolkits import mplot3d
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import plotnine as pn
import mizani.breaks as mzb
import mizani.formatters as mzf

from glob import glob
import seaborn as sns
blue, green, red, purple, gold, teal = sns.color_palette('colorblind', 6)
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

print()
%watermark -p pandas,pandas_datareader,numpy,scipy,matplotlib,seaborn,plotnine,dask,pyarrow,fastparquet

2017-11-22T19:57:09-07:00

CPython 3.6.2
IPython 6.1.0

compiler   : GCC 4.8.2 20140120 (Red Hat 4.8.2-15)
system     : Linux
release    : 4.10.0-38-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 12
interpreter: 64bit


/home/bcr/anaconda3/envs/bayesian/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools



pandas 0.21.0
pandas_datareader 0.5.0
numpy 1.13.1
scipy 1.0.0
matplotlib 2.1.0
seaborn 0.8.1
plotnine 0.3.0
dask 0.16.0
pyarrow 0.7.1
fastparquet 0.1.3


In [2]:
def cprint(df):
    print('-'*79)
    print('dataframe information')
    print('-'*79)
    print(df.tail(5))
    print('-'*50)
    print(df.info())
    print('-'*79)    
    print()

def lower_columns(df):
    """fn: to lower columns
    pipe func #1
    """
    df.columns = df.columns.str.lower()
    return df

def standardize_option_type_text(df):
    """fn: to lower option type ex. "Call" --> "call"
    pipe func #2
    """
    df['type'] = df['type'].str.lower()
    return df

# add spread and midpoint data to Yahoo
def compute_spread(df):
    """fn: to compute spread,midpoint,relative spread of bid-ask
    pipe func #3
    """    
    return (df
            .assign(spread = lambda df: df.ask - df.bid)
            .assign(mid_quote = lambda df: (df.ask + df.bid)/2)
            .assign(spread_pct = lambda df: df.spread / df.mid_quote))

# add instrinsic values
def compute_intrinsic(df):
    """fn: create intrinsic column
    pipe func #4
    """
    # create intrinsic value column
    call_q = 'type=="call"'
    call_intrinsic = ((df.query(call_q).loc[:, 'underlying_price'])
                      - (df.query(call_q).loc[:, 'strike']))
    put_q = 'type=="put"'
    put_intrinsic = ((df.query(put_q).loc[:, 'strike'])
                     - (df.query(put_q).loc[:, 'underlying_price']))
    
    # must drop down to pandas
    #  concat df sort index then set as column in dask df
    ci = call_intrinsic.compute() 
    pi = put_intrinsic.compute()
    intrinsic_series = pd.concat([ci, pi]).sort_index()    

    df = df.assign(intrinsic_value = intrinsic_series)
    return df

In [3]:
%%time


fp = data_dir+'intraday_options_data_2017-09-13_2017-10-18.parquet'

ddf = (dd.read_parquet(fp, engine='fastparquet')
       .pipe(lower_columns)
       .pipe(standardize_option_type_text)
       .pipe(compute_spread)
       .pipe(compute_intrinsic)
       .dropna(subset=['underlying_price','strike','type'])
       .reset_index(drop=True))
   

[########################################] | 100% Completed | 24.4s
[########################################] | 100% Completed | 38.9s
CPU times: user 43.4 s, sys: 13.9 s, total: 57.3 s
Wall time: 1min 43s


We can see how many data points we have below. 

In [4]:
n = delayed(len)(ddf)
m = delayed(len)(ddf.columns)
dd.compute(n*m)

[########################################] | 100% Completed | 31.1s


(321461074,)

Next up, I thought it would be helpful to calculate the _moneyness_ of each option (ex. at-the-money (ATM), in-the-money (ITM), out-of-money (OTM)) as a column in our dataframe. 

#### Doing this in pandas is a straightforward process when you can fit the entire dataset in-memory. To do this within the dask framework presents a challenge. Why? 

In `pandas` there are some assignment operations that change the df _in place_ and `dask` does not support these types of operations. My understanding is you run into all kinds of issues when trying to run tasks concurrently on a constantly mutating object. 

#### So how do we overcome this constraint?

We make clever use of the `dask.delayed` class which provides the flexibility to bring parallel computations to custom functions and loops. The intuition is that dask can do this by constructing a task graph with your functions in _lazy_ fashion, meaning it doesn't execute right away. When you call `dd.compute()` on the accumulated tasks/functions, it executes the task graph intelligently making use of dask's parallelization.

The functions we can use to do this are briefly explained below:
    
- `is_atm`, `is_itm`, `is_otm`: 
    - these functions take a dask dataframe, a tolerance value between 0 and 1 to act as the moneyness boundary, and the option type. 
    - these functions output the indexes containing the locations of the particular option status, (ATM, OTM, ITM)
    
    
- `make_pd_series`, `concat_sers`:
    - these functions are helper functions to the workhourse fuction defined as `add_status_flags()`
    - the purpose of these functions is to structure the algorithm loop in a way that it can make use of the `dask.delayed` class which we really want to use in order to take advantage of dask's task graph and parallelization capabilities.
    
    
- `add_status_flags`: this is the workhorse function for adding the option status flags and works like so:
    1. it splits the main `dask.dataframe` according to option types e.g. calls and puts.
    2. it puts the 2 dfs, the option type labels, the is_funcs(), and the option status labels into separate lists.
    3. it loops over the option type df list
        a. for each df, it loops over each option status function and its label
        b. it converts the computation of the indexes and the helper functions to `dask.delayed` objects to be executed later
        c. it accumulates the delayed functions/tasks in a list
    4. then it actually `dd.computes()` within the `concat_sers()` function which outputs the final `option_status` column as a `pd.Series` object.
    5. afterwards the function converts the series into a `pd.DataFrame` before it is converted into a `dask.dataframe`
    6. after the `dask.dataframe` conversion it is merged with the main `dask.dataframe`
    7. finally it iterates over the object columns converting them into the `categorical` dtype. Please note that for whatever reason a column with only `True, False` values converted to a categorical generates an error when you try to reimport the dataset in `parquet` format. As a result those columns must be converted to `bool` dtype.

In [5]:
# define ATM, ITM and OTM flag for each option type
def is_atm(df, tol, option_type):
    '''fn: add flag for options at the money as defined by user
    
    # Params
        df: dask.DataFrame
            options dataframe
        tol: float
            sets percentage band around strike price for inclusion
        option_type: str, (Call, Put)
    # Returns
        pd.Index: pandas index of atm flags
    '''
    if option_type=="Call" or option_type=='call':
        call_atm_ratio_lt = 1 + tol
        call_atm_ratio_gt = 1 - tol
        # Select contract using filters and constraints
        calls = df[df['type']==option_type]
        c1 = calls.underlying_price <= (calls.strike * call_atm_ratio_lt)
        c2 = calls.underlying_price >= (calls.strike * call_atm_ratio_gt)
        return calls.loc[c1 & c2].index  
    elif option_type=="Put" or option_type=='put':
        put_atm_ratio_lt = 1 + tol
        put_atm_ratio_gt = 1 - tol
        # Select contract using filters and constraints
        puts = df[df['type']==option_type]
        p1 = puts.underlying_price >= (puts.strike * put_atm_ratio_gt)
        p2 = puts.underlying_price <= (puts.strike * put_atm_ratio_lt)              
        return puts.loc[p1 & p2].index   
        
def is_itm(df, tol, option_type):
    '''fn: add flag for options in the money
    
    # Params
        df: dask.DataFrame
            options dataframe
        tol: float
            sets percentage band around strike price for inclusion
        option_type: str, (Call, Put)
    # Returns
        pd.Index: pandas index of itm flags
    '''
    if option_type=="Call" or option_type=='call':
        call_itm_ratio = 1 + tol
        # Select contract using filters and constraints
        calls = df[df['type']==option_type]
        c1 = calls.underlying_price > (calls.strike * call_itm_ratio)
        return calls.loc[c1].index  
    elif option_type=="Put" or option_type=='put':
        put_itm_ratio = 1 - tol
        # Select contract using filters and constraints
        puts = df[df['type']==option_type]
        p1 = puts.underlying_price < (puts.strike * put_itm_ratio)
        return puts.loc[p1].index
    
def is_otm(df, tol, option_type):
    '''fn: add flag for options out of the money
    
    # Params
        df: dask.DataFrame
            options dataframe
        tol: float
            sets percentage band around strike price for inclusion
        option_type: str, (Call, Put)
    # Returns
        pd.Index: pandas index of otm flags
    '''
    if option_type=="Call" or option_type=='call':
        call_otm_ratio = 1 - tol
        # Select contract using filters and constraints
        calls = df[df['type']==option_type] 
        c1 = calls.underlying_price < (calls.strike * call_otm_ratio)
        return calls.loc[c1].index  
    elif option_type=="Put" or option_type=='put':
        put_otm_ratio = 1 + tol
        # Select contract using filters and constraints
        puts = df[df['type']==option_type]
        p1 = puts.underlying_price > (puts.strike * put_otm_ratio)
        return puts.loc[p1].index  

def make_pd_series(tmp_df_index, status):
    """fn: helper to add_status_flags"""
    tmp_ser = pd.Series([status]*len(tmp_df_index),
                        index=tmp_df_index)
    return tmp_ser

def concat_sers(delayed_list, _range=None):
    """fn: helper to add_status_flags
    
    # Args
        delayed_list : dask delayed object
            list contains nested series
        _range : int()
            num of series contained in nested list
            
    # Returns
        cser : pd.Series
            concatenated series of index, status pairs
    """
    sers_list = dd.compute(delayed_list)
    cser = (pd.concat([sers_list[0][x] 
                     for x in range(_range)])
            .sort_index())
    return cser
    
def add_status_flags(data, tol):
    '''fn: add option moneyness flag based on tolerance
    
    # Params
        data: dask.DataFrame
            options dataframe
        tol: float
            sets percentage band around strike price
    # Returns
        df: dask.DataFrame
            df with option status column added
    '''    
    df = data #.copy()
    
    # must separate for memory issues
    calls = df[df['type']=='call']
    puts = df[df['type']=='put']    
    
    dfs = [calls, puts]
    types = ['call', 'put']
    status = ['ATM', 'ITM', 'OTM']
    funcs = [is_atm, is_itm, is_otm]
    
    delayed_sers = []
    for type_df, ot in tqdm(zip(dfs, types), desc='option type loop'):
        for s, fn in tqdm(zip(status, funcs), desc='status loop'): # zip status and funcs
            tmp_df_index = delayed(fn)(type_df, tol, ot) # get filtered temp df index
            dl_ser = delayed(make_pd_series)(tmp_df_index, s)
            delayed_sers.append(dl_ser)
    
    # concat series
    concat_ser = concat_sers(delayed_sers, _range=len(types)*len(status))
    concat_ser.name = 'option_status'
    
    # must convert series to frame and convert to dask object
    dd_concat_ser = dd.from_pandas(concat_ser.to_frame(), npartitions=1)
    # merge dask dfs
    return_df = df.merge(dd_concat_ser, left_index=True, right_index=True) 
    
    # convert object cols to categorical and bool
    obj_cols = return_df.select_dtypes(include=['object']).columns
    for col in obj_cols:
        # True False column as category creates error on read_parquet()
        if col=='isnonstandard': 
            return_df[col] = return_df[col].astype('bool')
        else:
            return_df[col] = return_df[col].astype('category')
        
    return return_df

Finally we execute the functions. 

In [6]:
tol = 0.05 # 5%
mdf = (add_status_flags(ddf, tol))

option type loop: 0it [00:00, ?it/s]
status loop: 0it [00:00, ?it/s]
status loop: 3it [00:00, 963.54it/s]
status loop: 0it [00:00, ?it/s]
option type loop: 2it [00:00, 187.29it/s]

[                                        ] | 0% Completed |  0.1s

[########################################] | 100% Completed | 44.7s


Just to give you an idea of the speedups I was able to achieve by implementing `dask.delayed` above, when I first wrote this code, I did it in `pandas` and it was sequential. The function took ~8 minutes to complete. Now it takes ~45 seconds. 
___

Below is a simple `save to parquet` function with some basic internal type checking.

In [7]:
def pq_save(fp, data):
    """fn: save to parquet
    
    # Args
        fp : filepath
        data : pd.DataFrame, dask.dataframe
    """
    if isinstance(data, pd.DataFrame):
        _table = pa.Table.from_pandas(data)
        pq.write_table(_table, fp)
    elif isinstance(data, dd.core.DataFrame):
        dd.to_parquet(fp, data,
                      engine='fastparquet',
                      compression='snappy')
    else:
        raise ValueError('data has wrong dtype')

Example save to disk as `parquet` format

In [8]:
file_title = 'intraday_options_data_master_2017-09-13_2017-10-18.parquet'
fp = data_dir + file_title

pq_save(fp, mdf)

[########################################] | 100% Completed |  1min  4.5s


Example of reading in `parquet` formatted data into a `dask.dataframe`

In [9]:
mdf = dd.read_parquet(fp)
mdf.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 23 entries, strike to option_status
dtypes: category(5), datetime64[ns](3), bool(1), float64(14)

Now we can code the actual computation logic. 

The `class` object below is called `etf_iv_skew`. It has `2` attributes, and `3` methods. It is initialized by the `data`, and `query_date` args.
- `data` is expected to be a `dask.dataframe` filtered to a singular symbol
- `query_date` is expected to be a `pandas.Timestamp` object. 
    
    - This date is the effective cutoff date between "today", the day of calculation, and 60 days from now which acts as a constraint on the tenor of the contracts we will analyze.    

The `3` methods consist of:
- `otm_put_max` : this function implements the selection of the appropriate OTM put contract for use in calculating the skew
- `atm_call_max` : this function implements the selection of the appropriate ATM call contract for use in calculating the skew
- `compute_iv_skew` : this implements the skew calculation which is essentially $otm\_put\_max - atm\_call\_max$.

In [10]:
class etf_iv_skew:
    """
    class for computing implied volatility skew
    
    # Attributes
        data : dask.dataframe()
            filtered ddf by symbol
        query_date : pd.Timestamp(), 
            date from which the t+60 calculation will take place
            
    # Methods
        otm_put_max : compute otm put max
        atm_call_max : compute atm call max
        compute_iv_skew : compute skew metric
    """
    def __init__(self, data, query_date=None):
        self.df = data
        
        if not query_date:
            self.query_date = pd.to_datetime('today').date()
        else:
            self.query_date = query_date
        
    def otm_put_max(self, option_type='put'):
        """fn: calc otm put max according to citation

        # Args
            data : dask.dataframe(), option data
            option_type : str(), (call, put)

        # Returns
            otm_put_max : pd.Series()
                contains max otm option info
        """

        t60 = self.query_date + 60 * pd.tseries.offsets.BDay()

        put_otm_ratio_lt = 0.95
        put_otm_ratio_gt = 0.91

        type_crit = f'(type=="{option_type}")'
        date_crit = f'(quote_time <= "{pd.to_datetime(self.query_date)}")'   
        expiry_crit = f'(expiry < "{t60.date()}")'
        put_q = ' & '.join([type_crit,date_crit,expiry_crit])
        sym_puts = self.df.query(put_q)

        strike_ratio = (sym_puts['strike'] / sym_puts['underlying_price'])
        put_lt_ratio_crit = put_otm_ratio_lt > strike_ratio
        put_gt_ratio_crit = strike_ratio > put_otm_ratio_gt

        otm_put_max = (sym_puts[(put_lt_ratio_crit) & (put_gt_ratio_crit)]
                       .max())
        return otm_put_max

    def atm_call_max(self, option_type='call'):
        """fn: calc atm put max according to citation

        # Args
            data : dask.dataframe(), option data
            option_type : str(), (call, put)

        # Returns
            atm_put_max : pd.Series()
                contains max atm option info
        """
        t60 = self.query_date + 60 * pd.tseries.offsets.BDay()

        call_atm_ratio_lt = 1.05
        call_atm_ratio_gt = 0.95

        type_crit = f'(type=="{option_type}")'
        date_crit = f'(quote_time <= "{pd.to_datetime(self.query_date)}")'        
        expiry_crit = f'(expiry < "{t60.date()}")'
        call_q = ' & '.join([type_crit,date_crit,expiry_crit])
        sym_calls = self.df.query(call_q)

        strike_ratio = (sym_calls['strike'] / sym_calls['underlying_price'])
        call_lt_ratio_crit = call_atm_ratio_lt > strike_ratio
        call_gt_ratio_crit = strike_ratio > call_atm_ratio_gt

        atm_call_max = (sym_calls[(call_lt_ratio_crit) & (call_gt_ratio_crit)]
                        .max())
        return atm_call_max    

    def compute_skew(self, otm_put, atm_call):
        """fn: compute option skew according to citation
        SKEW_{i, t} = VOLotmp_{i, t} - VOLatmc_{i, t}

        # Args
            otm_put : pd.Series()
                contains otm put max info
            atm_put : pd.Series()
                contains atm call max info
        # Returns
            skew measure : float()
        """
        return (otm_put['iv'] - atm_call['iv'])  

Below is a helper function that takes a dask.dataframe filtered on a symbol, and the interval frequency. By default the date interval is weekly. 

In [11]:
def dask_date_interval(ddf, interval='1W'):
    """fn: make date range"""
    start = ddf.quote_time.min()
    end = ddf.quote_time.max()
    
    dt_range = delayed(pd.date_range)(start, end, 
                                      freq=interval,
                                      normalize=True)
    return dt_range.shift(1)

We can run an example skew computation using `SPY` to confirm everything works before moving forward.

In [12]:
symbol = 'SPY'
spy_ddf = mdf.query(f'root=="{symbol}"')
dt_range = dask_date_interval(spy_ddf).compute()
print(dt_range)

[########################################] | 100% Completed |  8.2s
DatetimeIndex(['2017-09-24', '2017-10-01', '2017-10-08', '2017-10-15',
               '2017-10-22'],
              dtype='datetime64[ns]', freq='W-SUN')


In [13]:
iv = etf_iv_skew(spy_ddf, query_date=dt_range[0])

otm_put = delayed(iv.otm_put_max())
atm_call = delayed(iv.atm_call_max())

iv_skew = delayed(iv.compute_skew)(otm_put, atm_call)

print(f'{symbol} skew metric: {iv_skew.compute():0.4f}')

[########################################] | 100% Completed | 10.4s
SPY skew metric: 0.1211


The following functions are used to compute and aggregate the skew computations. 

- `run_skew`: this is a wrapper func to compute the skew for a single symbol and `query_date`
- `make_symbol_skew_df`: this func converts the symbol's `skew_dict` into a `pd.DataFrame`
- `make_skew_df`: this func aggregates the `all_syms` dict into two dfs
    - `raw_df` contains the raw skew values
    - `binned_df` contains the binned skews

In [14]:
def run_skew(ddf, query_date=None):
    """fn: wrapper for skew calc"""

    iv = etf_iv_skew(ddf, query_date=query_date)

    otm_put = delayed(iv.otm_put_max())    
    atm_call = delayed(iv.atm_call_max())
    
    iv_skew = delayed(iv.compute_skew)(otm_put, atm_call)
    
    return iv_skew

def make_symbol_skew_df(skews, symbol):
    """fn: to convert skew dict into df with stats"""
    skew_df = (pd.DataFrame()
               .from_dict(skews, orient='index')
               .rename(columns={0:symbol}))
    skew_df.index = pd.DatetimeIndex(skew_df.index)
    return skew_df

def make_skew_df(all_syms):
    """fn: to convert skew dict into df with stats
    
    # Args
        all_syms: dict(),
            keys:symbols; values:pd.DataFrame()
    # Returns
        concat_df: df containing raw values
        skew_df: decile df
    """
    raw_df = (pd.concat(list(all_syms.values()),
                        axis=1)
              .dropna(how='all', axis=1))
    binned_df = (raw_df
               .apply(pd.qcut, args=(10,),
                      labels=False, axis=1)) + 1
    return raw_df, binned_df


Finally we can implement these functions. I chose to compute the skews of the top 200 symbols based on average open interest.

The code loops through each symbol, extracts the start and ending `quote_time` and uses those to construct a weekly date range. Then the code loops through each date computing the skew. That data is collected in the `sym_skews` dict which uses the `query_date` as its keys and the skew metric as the values.

After all the dates are collected we run the `make_symbol_skew_df` and add that to the `all_syms` dict. We also save each symbol's `skew_df` to disk.

#### Note:

When I first coded this portion of the notebook I did the top 50 symbols, and was not making use of efficient looping or `dask.delayed`. It took `15 hours` to run on my machine. After refactoring the code it took ~`40 minutes`. Since the time savings were so significant I decided to compute the results for the top `200` instead. It was a reasonable ~`2 hours 40 minutes`. You can adjust the number of symbols analyzed by changing the `k` variable below.

In [17]:
%%time

# choose to calc etf with most open interest
k = 200
top_k = mdf.groupby(['root'])['open_int'].mean().nlargest(k).compute()
date_today = pd.to_datetime('today').date()

missing_symbols = {}
all_syms = {}

skew_dir = data_dir + '/skew_calcs/'

for sym in top_k.index: 
    sym_skews = {}    
    print()
    print('*'*77)   
    print(f'computing {sym}...')
    print()
    try:
        # get relevant quote dates by first filtering by symbol
        sym_ddf = mdf.query(f'root=="{sym}"')
        dt_range = dask_date_interval(sym_ddf).compute()
        print('n dates: ', len(dt_range))
        
        # iterate over dates in date range, compute skews, add to symbol dict
        for i, q_date in enumerate(dt_range):
            print('-'*50)
            print(f'computing {sym} {i} of {len(dt_range)} relative to query date: {q_date.date()}')               
            try:
                iv_skew = run_skew(sym_ddf, query_date=q_date)
                skew_metric = iv_skew.compute() # compute skew
                sym_skews[q_date.date()] = skew_metric # add to symbol dict
                print(f'{sym} {q_date.date()} skew: {skew_metric:0.4f}')
            except Exception as e:
                print(e)
                    
        # make skew df and add to all symbol dict
        skew_df = make_symbol_skew_df(sym_skews, sym)
        all_syms[sym] = skew_df
        
        # save data by symbol with the date of calculation (today) 
        #  all query dates are contained in each symbol's df
        _fp = skew_dir + f'{sym}_skew_data_{date_today}.parquet'
        pq_save(_fp, skew_df)     

    except Exception as e:
        print(f'{sym} error, {e}')
        missing_symbols[sym] = e   
        

[########################################] | 100% Completed |  7.0s

[                                        ] | 0% Completed |  0.0s*****************************************************************************
computing EEM...

[########################################] | 100% Completed |  7.3s
[                                        ] | 0% Completed |  0.0s 6
--------------------------------------------------
computing EEM 0 of 6 relative to query date: 2017-09-17
[########################################] | 100% Completed |  7.3s
[                                        ] | 0% Completed |  0.0s
--------------------------------------------------
computing EEM 1 of 6 relative to query date: 2017-09-24
[########################################] | 100% Completed |  8.7s
EEM 2017-09-24 skew: 0.4570
--------------------------------------------------
[                                        ] | 0% Completed |  0.0scomputing EEM 2 of 6 relative to query date: 2017-10-01
[#################

[########################################] | 100% Completed |  7.5s
[                                        ] | 0% Completed |  0.0s
--------------------------------------------------
computing UNG 3 of 5 relative to query date: 2017-10-15
[########################################] | 100% Completed |  7.5s
UNG 2017-10-15 skew: 0.3438
--------------------------------------------------
computing UNG 4 of 5 relative to query date: 2017-10-22
[########################################] | 100% Completed |  7.8s
UNG 2017-10-22 skew: 0.3438

*****************************************************************************
computing XOP...
[                                        ] | 0% Completed |  0.0s
[########################################] | 100% Completed |  7.4s
n dates:  5
[                                        ] | 0% Completed |  0.0s--------------------------------------------------
computing XOP 0 of 5 relative to query date: 2017-09-24
[########################################] | 1

[########################################] | 100% Completed |  7.6s
[                                        ] | 0% Completed |  0.0s
--------------------------------------------------
computing XLU 1 of 5 relative to query date: 2017-10-01
[########################################] | 100% Completed |  7.8s
[                                        ] | 0% Completed |  0.0s
--------------------------------------------------
computing XLU 2 of 5 relative to query date: 2017-10-08
[########################################] | 100% Completed |  7.7s
XLU 2017-10-08 skew: -1.6777
--------------------------------------------------
[                                        ] | 0% Completed |  0.0scomputing XLU 3 of 5 relative to query date: 2017-10-15
[########################################] | 100% Completed |  8.0s
XLU 2017-10-15 skew: -1.6777
--------------------------------------------------
[                                        ] | 0% Completed |  0.0s
[##################################

[########################################] | 100% Completed |  7.4s
EWA 2017-10-22 skew: 0.5117

*****************************************************************************
computing FEZ...
[                                        ] | 0% Completed |  0.0s
[########################################] | 100% Completed |  7.4s
n dates:  5
[                                        ] | 0% Completed |  0.0s--------------------------------------------------
computing FEZ 0 of 5 relative to query date: 2017-09-24
[########################################] | 100% Completed |  7.6s
FEZ 2017-09-24 skew: 1.2187
--------------------------------------------------
[                                        ] | 0% Completed |  0.0scomputing FEZ 1 of 5 relative to query date: 2017-10-01
[########################################] | 100% Completed |  7.8s
FEZ 2017-10-01 skew: 1.2187
--------------------------------------------------
[                                        ] | 0% Completed |  0.0scomputing 

[########################################] | 100% Completed |  7.8s
TLT 2017-10-08 skew: -1.1289
[                                        ] | 0% Completed |  0.0s
computing TLT 3 of 5 relative to query date: 2017-10-15
[########################################] | 100% Completed |  8.0s
TLT 2017-10-15 skew: -1.0820
[                                        ] | 0% Completed |  0.0s
computing TLT 4 of 5 relative to query date: 2017-10-22
[########################################] | 100% Completed |  8.1s
TLT 2017-10-22 skew: -1.0820

*****************************************************************************
computing EWJ...
[                                        ] | 0% Completed |  0.0s
[########################################] | 100% Completed |  7.3s
n dates:  5
[                                        ] | 0% Completed |  0.0s
computing EWJ 0 of 5 relative to query date: 2017-09-24
[########################################] | 100% Completed |  7.7s
EWJ 2017-09-24 skew: 0.6289
-----

...

[########################################] | 100% Completed |  7.8s
VTI 2017-10-08 skew: 0.4746
--------------------------------------------------
computing VTI 3 of 5 relative to query date: 2017-10-15
[########################################] | 100% Completed |  7.5s
VTI 2017-10-15 skew: 0.4746
--------------------------------------------------
[                                        ] | 0% Completed |  0.0scomputing VTI 4 of 5 relative to query date: 2017-10-22
[########################################] | 100% Completed |  7.5s
VTI 2017-10-22 skew: 0.4746

*****************************************************************************
computing SEA...
[                                        ] | 0% Completed |  0.0s
[########################################] | 100% Completed |  7.4s
[                                        ] | 0% Completed |  0.0s 5
--------------------------------------------------
computing SEA 0 of 5 relative to query date: 2017-09-24
[###########################

[########################################] | 100% Completed |  8.9s
n dates:  5
--------------------------------------------------
[                                        ] | 0% Completed |  0.0scomputing IJR 0 of 5 relative to query date: 2017-09-24
[########################################] | 100% Completed |  8.8s
[                                        ] | 0% Completed |  0.0s
--------------------------------------------------
computing IJR 1 of 5 relative to query date: 2017-10-01
[########################################] | 100% Completed |  8.6s
IJR 2017-10-01 skew: 0.3340
[                                        ] | 0% Completed |  0.0s
computing IJR 2 of 5 relative to query date: 2017-10-08
[########################################] | 100% Completed |  8.5s
[                                        ] | 0% Completed |  0.0s
--------------------------------------------------
computing IJR 3 of 5 relative to query date: 2017-10-15
[########################################] | 100

computing IWF...

[########################################] | 100% Completed |  7.3s
n dates:  5
[                                        ] | 0% Completed |  0.0s
computing IWF 0 of 5 relative to query date: 2017-09-24
[########################################] | 100% Completed |  7.2s
[                                        ] | 0% Completed |  0.0s
--------------------------------------------------
computing IWF 1 of 5 relative to query date: 2017-10-01
[########################################] | 100% Completed |  7.4s
IWF 2017-10-01 skew: 0.3076
--------------------------------------------------
computing IWF 2 of 5 relative to query date: 2017-10-08
[########################################] | 100% Completed |  7.7s
IWF 2017-10-08 skew: 0.3076
--------------------------------------------------
computing IWF 3 of 5 relative to query date: 2017-10-15
[########################################] | 100% Completed |  7.5s
[                                        ] | 0% Completed |  0.0s

[########################################] | 100% Completed |  8.7s
BZF 2017-10-15 skew: 0.9258
[                                        ] | 0% Completed |  0.0s--------------------------------------------------
computing BZF 4 of 5 relative to query date: 2017-10-22
[########################################] | 100% Completed |  7.6s
BZF 2017-10-22 skew: 0.8242

*****************************************************************************
computing RUSL...
[                                        ] | 0% Completed |  0.0s
[########################################] | 100% Completed |  7.3s
n dates:  5
[                                        ] | 0% Completed |  0.0s
computing RUSL 0 of 5 relative to query date: 2017-09-24
[########################################] | 100% Completed |  7.3s
RUSL 2017-09-24 skew: 0.2813
[                                        ] | 0% Completed |  0.0s
computing RUSL 1 of 5 relative to query date: 2017-10-01
[########################################] | 100% 

[########################################] | 100% Completed |  7.4s
HEZU 2017-10-01 skew: 0.3022
--------------------------------------------------
[                                        ] | 0% Completed |  0.0s
[########################################] | 100% Completed |  7.3s
HEZU 2017-10-08 skew: 0.3657
--------------------------------------------------
computing HEZU 3 of 5 relative to query date: 2017-10-15
[########################################] | 100% Completed |  7.5s
HEZU 2017-10-15 skew: 0.3999
[                                        ] | 0% Completed |  0.0s--------------------------------------------------
computing HEZU 4 of 5 relative to query date: 2017-10-22
[########################################] | 100% Completed |  7.5s
HEZU 2017-10-22 skew: 0.3936

*****************************************************************************
[                                        ] | 0% Completed |  0.0s

[########################################] | 100% Completed |  7.0s


Finally we aggregate all the data into dataframes and save those to disk as `parquet` files. 

In [18]:
raw_df, binned_df = make_skew_df(all_syms)

In [19]:
cprint(raw_df)

-------------------------------------------------------------------------------
dataframe information
-------------------------------------------------------------------------------
              EEM    USO     XLF     SPY    GDX    UNG     XOP    EFA    AAXJ  \
2017-09-24 0.4570 0.2812  0.1953  0.1211 0.2266 0.3516 -0.0391 0.1504 -0.7393   
2017-10-01 0.4570 0.3125  0.1953 -0.4370 0.1875 0.4766 -0.0391 0.1504 -0.7393   
2017-10-08 0.4570 0.1758  0.2734 -0.4370 0.1719 0.3438  0.0547 0.1504 -0.7393   
2017-10-15 0.4570 0.1758 -0.2070 -0.4331 0.1719 0.3438  0.0547 0.1973 -0.7393   
2017-10-22 0.4570 0.1758 -0.2070 -0.4331 0.1719 0.3438  0.0547 0.1973 -0.7393   

              EWZ  ...     DBEF     SKF    IVV    TECS     SOXS    BIB   HEZU  \
2017-09-24 0.1953  ...   0.0708 -2.1963 0.1533     nan -10.9609 0.0332 0.2436   
2017-10-01 0.1328  ...   0.1001 -2.1963 0.1533 -1.0527 -10.9609 0.0332 0.3022   
2017-10-08 0.1328  ...   0.1533 -2.1094 0.1533 -1.0527 -10.9609 0.0332 0.3657   
2017-10

In [20]:
cprint(binned_df)

-------------------------------------------------------------------------------
dataframe information
-------------------------------------------------------------------------------
              EEM    USO    XLF    SPY    GDX    UNG    XOP    EFA   AAXJ  \
2017-09-24 8.0000 7.0000 6.0000 5.0000 6.0000 7.0000 4.0000 5.0000 2.0000   
2017-10-01 8.0000 7.0000 6.0000 3.0000 6.0000 8.0000 4.0000 6.0000 2.0000   
2017-10-08 8.0000 6.0000 7.0000 3.0000 6.0000 8.0000 5.0000 6.0000 2.0000   
2017-10-15 8.0000 6.0000 4.0000 3.0000 6.0000 7.0000 5.0000 6.0000 2.0000   
2017-10-22 8.0000 6.0000 4.0000 3.0000 6.0000 7.0000 5.0000 6.0000 2.0000   

              EWZ  ...     DBEF    SKF    IVV   TECS   SOXS    BIB   HEZU  \
2017-09-24 6.0000  ...   5.0000 1.0000 5.0000    nan 1.0000 5.0000 6.0000   
2017-10-01 5.0000  ...   5.0000 1.0000 6.0000 2.0000 1.0000 5.0000 7.0000   
2017-10-08 5.0000  ...   6.0000 1.0000 6.0000 2.0000 1.0000 5.0000 8.0000   
2017-10-15 5.0000  ...   6.0000 1.0000 5.0000 2

In [21]:
def save_raw_df(raw_df):
    _fp = skew_dir + f'agg_raw_skew_data_{date_today}.parquet'
    pq_save(_fp, raw_df)
    
def save_binned_df(binned_df):
    _fp = skew_dir + f'agg_binned_skew_data_{date_today}.parquet'
    pq_save(_fp, binned_df)   

In [23]:
save_raw_df(raw_df)
save_binned_df(binned_df)